In [ ]:
# Install required packages
# Uncomment and run this cell if you haven't installed these packages
# !pip install --quiet ultralytics opencv-python matplotlib torch torchvision tqdm
# Note: For CUDA support, you might need to install specific torch versions
# Visit https://pytorch.org/get-started/locally/ for installation instructions

In [ ]:
# --- Import Required Libraries ---

# File and data handling
import os
import json
import random
import shutil
from collections import defaultdict
from pathlib import Path

# Data processing and visualization
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import yaml

# Deep learning
import torch
from ultralytics import YOLO
from tqdm.notebook import tqdm  # For progress bars

# Print pytorch version and cuda availability
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

<Logger detectron2 (DEBUG)>

In [ ]:
# --- 1. Configuration Section ---

# Paths to your data and model files
JSON_DIR = 'MangaSegmentation/jsons_processed'
IMAGE_ROOT_DIR = 'Manga109_released_2023_12_07/Manga109_released_2023_12_07/images'
DATASET_DIR = 'balloon_dataset'  # Directory to store YOLO format dataset
PRE_TRAINED_MODEL_DIR = 'pre-trained_model'

# Validate paths
for path in [JSON_DIR, IMAGE_ROOT_DIR]:
    if not os.path.exists(path):
        raise FileNotFoundError(f"Directory not found: {path}")

# Create dataset directories
print("Creating dataset directories...")
for split in ['train', 'val']:
    for subdir in ['images', 'labels']:
        dir_path = os.path.join(DATASET_DIR, f'{subdir}/{split}')
        os.makedirs(dir_path, exist_ok=True)
        print(f"Created: {dir_path}")

# The category we want to train on
TARGET_CATEGORY_ID = 5
TARGET_CATEGORY_NAME = "balloon"

In [ ]:
# --- 2. Data Preparation ---

def xywh_to_yolo(bbox, img_width, img_height):
    """Convert XYWH bbox to YOLO format (x_center, y_center, width, height) normalized"""
    # Validate input
    if any(not isinstance(x, (int, float)) for x in bbox):
        raise ValueError("Bbox coordinates must be numeric")
    if img_width <= 0 or img_height <= 0:
        raise ValueError("Image dimensions must be positive")
    
    x, y, w, h = bbox
    # Validate bbox dimensions
    if w <= 0 or h <= 0:
        raise ValueError("Bbox width and height must be positive")
    if x < 0 or y < 0 or x + w > img_width or y + h > img_height:
        raise ValueError("Bbox coordinates out of image bounds")
    
    return [
        (x + w/2) / img_width,  # x_center
        (y + h/2) / img_height, # y_center
        w / img_width,          # width
        h / img_height          # height
    ]

def prepare_manga_balloon_data(json_dir, image_root):
    """
    Loads pre-processed JSON files and filters for the target category.
    Returns a dictionary of image paths and their corresponding annotations.
    """
    if not os.path.exists(json_dir):
        raise FileNotFoundError(f"JSON directory not found: {json_dir}")
    if not os.path.exists(image_root):
        raise FileNotFoundError(f"Image directory not found: {image_root}")
    
    dataset_dicts = {}
    all_images = {}
    all_annotations = defaultdict(list)
    
    print("Loading and parsing PRE-PROCESSED JSON files...")
    json_files = [f for f in os.listdir(json_dir) if f.endswith('.json')]
    if not json_files:
        raise FileNotFoundError(f"No JSON files found in {json_dir}")
    
    for json_file in json_files:
        try:
            with open(os.path.join(json_dir, json_file), 'r') as f:
                data = json.load(f)
                for img_info in data['images']:
                    all_images[img_info['id']] = img_info
                for ann_info in data['annotations']:
                    if ann_info['category_id'] == TARGET_CATEGORY_ID:
                        all_annotations[ann_info['image_id']].append(ann_info)
        except json.JSONDecodeError:
            print(f"Warning: Failed to parse {json_file}")
            continue

    print(f"Loaded data for {len(all_images)} total images.")
    
    for img_id, img_info in all_images.items():
        if img_id in all_annotations:  # Only include images that have balloon annotations
            img_path = os.path.join(image_root, img_info['file_name'])
            if not os.path.exists(img_path):
                print(f"Warning: Image not found: {img_path}")
                continue
                
            width = img_info['width']
            height = img_info['height']
            
            # Convert annotations to YOLO format
            yolo_bboxes = []
            for ann in all_annotations[img_id]:
                try:
                    yolo_bbox = xywh_to_yolo(ann['bbox'], width, height)
                    yolo_bboxes.append(yolo_bbox)
                except (ValueError, KeyError) as e:
                    print(f"Warning: Invalid bbox in {img_path}: {e}")
                    continue
            
            if yolo_bboxes:  # Only include if valid annotations exist
                dataset_dicts[img_path] = {
                    'image_id': img_id,
                    'width': width,
                    'height': height,
                    'bboxes': yolo_bboxes
                }
            
    if not dataset_dicts:
        raise ValueError("No valid data found after processing")
            
    print(f"Finished data preparation. Found {len(dataset_dicts)} images containing '{TARGET_CATEGORY_NAME}'.")
    return dataset_dicts

In [ ]:
# --- 3. Split and Prepare YOLO Dataset ---

# Prepare the data
all_data = prepare_manga_balloon_data(JSON_DIR, IMAGE_ROOT_DIR)

# --- Group data by manga title ---
print("\nGrouping data by manga series for a robust train/val split...")
grouped_data = defaultdict(dict)
for img_path, data in all_data.items():
    # Extract manga name from the file path
    manga_name = Path(img_path).parent.name
    grouped_data[manga_name][img_path] = data

print(f"Found {len(grouped_data)} unique manga series.")

# --- Split manga titles, not individual pages ---
manga_titles = list(grouped_data.keys())
train_titles, val_titles = train_test_split(manga_titles, test_size=0.2, random_state=42)

print(f"Splitting into {len(train_titles)} series for training and {len(val_titles)} for validation.")

# Function to copy images and create label files
def process_dataset_split(titles, split_type):
    for manga_title in titles:
        for img_path, data in grouped_data[manga_title].items():
            # Copy image
            dest_img_path = os.path.join(DATASET_DIR, f'images/{split_type}', 
                                       f"{manga_title}_{Path(img_path).name}")
            shutil.copy2(img_path, dest_img_path)
            
            # Create label file
            label_filename = Path(dest_img_path).stem + '.txt'
            label_path = os.path.join(DATASET_DIR, f'labels/{split_type}', label_filename)
            
            # Write YOLO format labels
            with open(label_path, 'w') as f:
                for bbox in data['bboxes']:
                    # Class index (0 for balloon) followed by bbox coordinates
                    f.write(f"0 {' '.join(map(str, bbox))}\n")

# Process train and validation splits
print("\nProcessing training split...")
process_dataset_split(train_titles, 'train')
print("Processing validation split...")
process_dataset_split(val_titles, 'val')

# Verify split
train_images = len(list(Path(DATASET_DIR).glob('images/train/*.jpg')))
val_images = len(list(Path(DATASET_DIR).glob('images/val/*.jpg')))
print(f"\nDataset created successfully:")
print(f"Training images: {train_images}")
print(f"Validation images: {val_images}")

# Save split information
split_info = pd.DataFrame([
    {'manga_title': title, 'dataset_split': 'train'} for title in train_titles
] + [
    {'manga_title': title, 'dataset_split': 'validation'} for title in val_titles
])
split_info.sort_values('manga_title').to_csv('manga_split_summary.csv', index=False)

Loading and parsing PRE-PROCESSED JSON files...
Loaded data for 10619 total images.
Finished data preparation. Found 9916 images containing 'balloon'.

Grouping data by manga series for a robust train/val split...
Found 109 unique manga series.
Splitting into 87 series for training and 22 for validation.

Final training set size: 8007 images
Final validation set size: 1909 images
Split verified: No data leakage between train and validation sets.


In [ ]:
# ===================================================================
# --- Create, Display, and Save Split Summary ---
# ===================================================================
print("\n--- Generating Train/Validation Split Summary ---")

# Create a list of dictionaries for the DataFrame
train_split_info = [{'manga_title': title, 'dataset_split': 'train'} for title in train_titles]
val_split_info = [{'manga_title': title, 'dataset_split': 'validation'} for title in val_titles]

# Combine and create the DataFrame
split_summary_df = pd.DataFrame(train_split_info + val_split_info)
split_summary_df = split_summary_df.sort_values(by='manga_title').reset_index(drop=True)

# Display the DataFrame in the notebook output
print("Manga Series Split Distribution:")
print(split_summary_df.to_string()) # .to_string() ensures all rows are printed

# Save the DataFrame to a CSV file
csv_filename = "manga_split_summary.csv"
split_summary_df.to_csv(csv_filename, index=False)
print(f"\nSplit summary has been saved to '{csv_filename}'")
# ===================================================================


--- Generating Train/Validation Split Summary ---
Manga Series Split Distribution:
                          manga_title dataset_split
0                                ARMS    validation
1                  AisazuNihaIrarenai         train
2                    AkkeraKanjinchou         train
3                             Akuhamu         train
4                        AosugiruHaru    validation
5                       AppareKappore         train
6                               Arisa         train
7                           BEMADER_P         train
8                 BakuretsuKungFuGirl         train
9                            Belmondo         train
10                  BokuHaSitatakaKun    validation
11            BurariTessenTorimonocho    validation
12                        ByebyeC-BOY    validation
13                Count3DeKimeteAgeru         train
14                            DollGun         train
15                       Donburakokko         train
16                        DualJu

In [ ]:
# --- 4. Create YAML Configuration ---

# Define dataset configuration
dataset_config = {
    # Đường dẫn dataset
    'path': os.path.abspath(DATASET_DIR),  # Đường dẫn tuyệt đối đến thư mục dataset
    'train': 'images/train',               # Thư mục ảnh training
    'val': 'images/val',                   # Thư mục ảnh validation
    'test': 'images/val',                  # Sử dụng validation set làm test set
    
    # Thông tin classes
    'names': {
        0: TARGET_CATEGORY_NAME  # Class 0: balloon
    },
    
    # Thông số task
    'task': 'detect',           # Task: object detection
    'nc': 1,                    # Số lượng class: 1 (balloon)
    
    # Thông tin thêm
    'width': 1280,              # Kích thước ảnh đầu vào
    'height': 1280,             # Giữ tỷ lệ 1:1 để tối ưu
    'batch': 8,                 # Batch size khuyến nghị
    'epochs': 100,              # Số epochs khuyến nghị
    'device': 0,                # GPU device index
}

# Save configuration
yaml_path = os.path.join(DATASET_DIR, 'dataset.yaml')
with open(yaml_path, 'w') as f:
    yaml.dump(dataset_config, f, default_flow_style=False, sort_keys=False)

print(f"Dataset configuration saved to {yaml_path}")
print("\nDataset configuration:")
print(yaml.dump(dataset_config, default_flow_style=False, sort_keys=False))


Configuring the model for training...


In [ ]:
# --- 5. Initialize and Train YOLOv8 Model ---

# Kiểm tra GPU
print("Checking GPU availability...")
import torch
if torch.cuda.is_available():
    print(f"Found {torch.cuda.device_count()} GPU(s)")
    print(f"Using: {torch.cuda.get_device_name(0)}")
else:
    print("WARNING: No GPU found. Training on CPU will be very slow!")

# Initialize YOLOv8 model
print("\nInitializing YOLOv8x model...")
model = YOLO('yolov8x.pt')  # Phiên bản lớn nhất và chính xác nhất

# Cấu hình training cơ bản
train_args = {
    # [1] Cấu hình dataset
    'data': yaml_path,         # File cấu hình dataset
    
    # [2] Các tham số training chính
    'epochs': 100,            # Số epochs training
    'imgsz': 1280,           # Kích thước ảnh đầu vào
    'batch': 8,              # Batch size (giảm nếu GPU yếu)
    'patience': 15,          # Early stopping nếu không cải thiện
    
    # [3] Tối ưu hóa
    'lr0': 0.001,           # Learning rate ban đầu
    'lrf': 0.01,            # Learning rate cuối
    'weight_decay': 0.0005, # Chống overfitting
    
    # [4] Data augmentation cơ bản
    'fliplr': 0.5,          # Lật ngang (balloon ở cả 2 bên)
    'scale': 0.3,           # Thay đổi kích thước (±30%)
    
    # [5] Thư mục và checkpoint
    'project': 'runs/detect',     # Thư mục gốc
    'name': 'balloon_yolov8',     # Tên experiment
    'exist_ok': True,             # Ghi đè thư mục cũ
    'pretrained': True,           # Dùng pretrained weights
    'device': 0 if torch.cuda.is_available() else 'cpu',  # Tự chọn device
}

print("\nTraining configuration:")
for section, params in {
    'Dataset': ['data'],
    'Training': ['epochs', 'imgsz', 'batch', 'patience'],
    'Optimization': ['lr0', 'lrf', 'weight_decay'],
    'Augmentation': ['fliplr', 'scale'],
    'Output': ['project', 'name']
}.items():
    print(f"\n{section}:")
    for p in params:
        print(f"  {p}: {train_args[p]}")

# Cảnh báo và lưu ý
print("\nNOTES:")
print("- Nếu GPU yếu, hãy giảm batch_size và imgsz")
print("- Có thể thêm augmentation nếu độ chính xác thấp")
print("- Early stopping sẽ dừng sau 15 epochs không cải thiện")
print("- Model sẽ tự lưu checkpoint tốt nhất")

[10/04 19:03:57 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'proposal_generator.rpn_head.objectness_logits.weight' to the model due to incompatible shapes: (3, 256, 1, 1) in the checkpoint but (5, 256, 1, 1) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.rpn_head.objectness_logits.bias' to the model due to incompatible shapes: (3,) in the checkpoint but (5,) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.rpn_head.anchor_deltas.weight' to the model due to incompatible shapes: (12, 256, 1, 1) in the checkpoint but (20, 256, 1, 1) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.rpn_head.anchor_deltas.bias' to the model due to incompatible shapes: (12,) in the checkpoint but (20,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.0.cls_score.weight' to the model due to incompatible

In [ ]:
# --- Start Training ---

print("Starting training process...")
print(f"Training will run for {train_args['epochs']} epochs")
print(f"Early stopping patience: {train_args['patience']} epochs")
print(f"Results will be saved to: {os.path.join(train_args['project'], train_args['name'])}")

# Training
from tqdm.notebook import tqdm
with tqdm(total=train_args['epochs'], desc='Training Progress') as pbar:
    def on_train_epoch_end(trainer):
        pbar.update(1)
    
    results = model.train(
        **train_args,
        callbacks=[on_train_epoch_end]
    )

# Print detailed results summary
print("\n=== Training Completed ===")
metrics = results.results_dict
print("\nAccuracy Metrics:")
print(f"mAP50-95: {metrics['metrics/mAP50-95(B)']:.4f} (primary metric)")
print(f"mAP50: {metrics['metrics/mAP50(B)']:.4f}")
print(f"Precision: {metrics['metrics/precision(B)']:.4f}")
print(f"Recall: {metrics['metrics/recall(B)']:.4f}")

print("\nTraining Statistics:")
print(f"Best epoch: {results.best_epoch}")
print(f"Final epoch: {results.epoch}")
if results.epoch < train_args['epochs']:
    print("Note: Training stopped early due to patience criterion")

# Save training plots
results.plot_results()
print("\nTraining plots have been saved to the project directory")

Starting training...
[10/04 19:03:58 d2.engine.train_loop]: Starting training from iteration 0


/home/zendragonxxx/.local/lib/python3.12/site-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[10/04 19:04:14 d2.utils.events]:  eta: 0:46:59  iter: 19  total_loss: 4.139  loss_cls_stage0: 0.8045  loss_box_reg_stage0: 0.08362  loss_cls_stage1: 0.5796  loss_box_reg_stage1: 0.208  loss_cls_stage2: 0.5901  loss_box_reg_stage2: 0.2377  loss_mask: 0.6914  loss_rpn_cls: 0.7742  loss_rpn_loc: 0.07407    time: 0.6910  last_time: 0.7113  data_time: 0.0733  last_data_time: 0.0011   lr: 1.6395e-05  max_mem: 5019M


2025-10-04 19:04:14.489470: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-04 19:04:14.499120: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-04 19:04:14.512056: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-04 19:04:14.515557: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-04 19:04:14.525054: I tensorflow/core/platform/cpu_feature_guar

[10/04 19:04:30 d2.utils.events]:  eta: 0:46:55  iter: 39  total_loss: 3.027  loss_cls_stage0: 0.3488  loss_box_reg_stage0: 0.08062  loss_cls_stage1: 0.3217  loss_box_reg_stage1: 0.2104  loss_cls_stage2: 0.3016  loss_box_reg_stage2: 0.2516  loss_mask: 0.6384  loss_rpn_cls: 0.7239  loss_rpn_loc: 0.07615    time: 0.7051  last_time: 0.7760  data_time: 0.0030  last_data_time: 0.0010   lr: 3.2776e-05  max_mem: 5315M
[10/04 19:04:44 d2.utils.events]:  eta: 0:47:46  iter: 59  total_loss: 2.333  loss_cls_stage0: 0.1922  loss_box_reg_stage0: 0.06893  loss_cls_stage1: 0.1968  loss_box_reg_stage1: 0.1719  loss_cls_stage2: 0.1817  loss_box_reg_stage2: 0.1766  loss_mask: 0.5392  loss_rpn_cls: 0.6142  loss_rpn_loc: 0.09681    time: 0.7152  last_time: 0.7727  data_time: 0.0030  last_data_time: 0.0010   lr: 4.9157e-05  max_mem: 5429M
[10/04 19:04:59 d2.utils.events]:  eta: 0:47:21  iter: 79  total_loss: 2.098  loss_cls_stage0: 0.1864  loss_box_reg_stage0: 0.06934  loss_cls_stage1: 0.187  loss_box_reg_

In [ ]:
# --- 6. Model Evaluation ---
print("\n=== Starting Model Evaluation ===")

# Validate on validation set
print("\nRunning validation...")
val_results = model.val(
    data=yaml_path,
    split='val',
    conf=0.25,  # Confidence threshold
    iou=0.45,   # NMS IOU threshold
    verbose=True
)

# Print detailed metrics
metrics = val_results.results_dict
print("\nDetailed Evaluation Metrics:")
print("1. Average Precision (AP):")
print(f"  - mAP50-95: {metrics['metrics/mAP50-95']:.4f}")
print(f"  - mAP50: {metrics['metrics/mAP50']:.4f}")
print(f"  - mAP75: {metrics['metrics/mAP75']:.4f}")

print("\n2. Precision/Recall:")
print(f"  - Precision: {metrics['metrics/precision']:.4f}")
print(f"  - Recall: {metrics['metrics/recall']:.4f}")

print("\n3. Performance:")
print(f"  - Speed: {metrics.get('speed/inference', 'N/A')} ms per image")
print(f"  - Total images: {val_results.need}")
print(f"  - Processed images: {val_results.done}")

# Plot confusion matrix
val_results.plot_confusion_matrix()
print("\nConfusion matrix has been saved to the project directory")


--- Setting up for testing ---
Found best checkpoint: Iteration=3999 with segm/AP=83.7070
Loading best model weights from: ./output_balloon_segmentation_v2/model_0003999.pth
Copied best model to: ./output_balloon_segmentation_v2/model_best.pth
[10/04 21:07:27 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output_balloon_segmentation_v2/model_0003999.pth ...
Predictor loaded successfully.


In [ ]:
# --- 7. Visualize Predictions ---
print("\n=== Generating Visualizations ===")

# Create directories
vis_output_dir = "./balloon_test_visualizations"
os.makedirs(vis_output_dir, exist_ok=True)

# Get validation images
val_images_dir = os.path.join(DATASET_DIR, 'images/val')
val_images = list(Path(val_images_dir).glob('*.jpg'))
print(f"\nFound {len(val_images)} validation images")

# Select random samples
num_samples = min(20, len(val_images))
sample_images = random.sample(val_images, num_samples)

print(f"\nProcessing {num_samples} random samples...")
for i, img_path in enumerate(sample_images, 1):
    print(f"\nImage {i}/{num_samples}: {img_path.name}")
    
    # Run prediction
    results = model.predict(
        source=str(img_path),
        save=True,          # Save annotated images
        save_txt=True,      # Save predictions as txt
        conf=0.25,          # Confidence threshold
        iou=0.45,          # NMS IOU threshold
        line_width=2,       # Box thickness
        boxes=True,         # Show boxes
        labels=True,        # Show labels
        hide_conf=False     # Show confidences
    )
    
    # Print detection summary
    r = results[0]  # Get first (only) result
    print(f"Detections: {len(r)} balloons")
    if len(r) > 0:
        print("Confidence scores:", [f"{conf:.2f}" for conf in r.boxes.conf])
    
    # Copy visualization
    pred_img = Path(r.save_dir) / img_path.name
    if pred_img.exists():
        dest_path = os.path.join(vis_output_dir, img_path.name)
        shutil.copy2(pred_img, dest_path)
        print(f"Saved visualization to: {dest_path}")

print(f"\nAll visualizations saved to '{vis_output_dir}'")
print("Note: Green boxes show predicted balloons with confidence scores")


--- Starting Qualitative Visualization on Random Validation Samples ---
Processing sample 1/20: Manga='LoveHina_vol14', Page='011.jpg'
Processing sample 2/20: Manga='TsubasaNoKioku', Page='023.jpg'
Processing sample 3/20: Manga='BokuHaSitatakaKun', Page='037.jpg'
Processing sample 4/20: Manga='PrayerHaNemurenai', Page='059.jpg'
Processing sample 5/20: Manga='ARMS', Page='054.jpg'
Processing sample 6/20: Manga='SaladDays_vol18', Page='064.jpg'
Processing sample 7/20: Manga='SaladDays_vol18', Page='025.jpg'
Processing sample 8/20: Manga='EvaLady', Page='047.jpg'
Processing sample 9/20: Manga='BokuHaSitatakaKun', Page='087.jpg'
Processing sample 10/20: Manga='EvaLady', Page='012.jpg'
Processing sample 11/20: Manga='PikaruGenkiDesu', Page='081.jpg'
Processing sample 12/20: Manga='BurariTessenTorimonocho', Page='049.jpg'
Processing sample 13/20: Manga='PikaruGenkiDesu', Page='039.jpg'
Processing sample 14/20: Manga='KyokugenCyclone', Page='039.jpg'
Processing sample 15/20: Manga='ByebyeC-B

In [ ]:
# --- 8. Export and Save Model ---
print("\n=== Exporting Model ===")

# 1. Save PyTorch model
best_model_path = 'balloon_detector_best.pt'
shutil.copy2(str(model.best), best_model_path)
print(f"\n1. PyTorch model saved as '{best_model_path}'")

# 2. Export to ONNX
print("\n2. Exporting to ONNX format...")
model.export(format='onnx', dynamic=True)
print("ONNX model exported successfully")

# 3. Export to TensorRT (if supported)
if torch.cuda.is_available():
    try:
        print("\n3. Exporting to TensorRT format...")
        model.export(format='engine', dynamic=True)
        print("TensorRT model exported successfully")
    except Exception as e:
        print(f"TensorRT export failed: {e}")

# 4. Export to CoreML (if on MacOS)
if os.uname().sysname == 'Darwin':
    try:
        print("\n4. Exporting to CoreML format...")
        model.export(format='coreml', dynamic=True)
        print("CoreML model exported successfully")
    except Exception as e:
        print(f"CoreML export failed: {e}")

# Save training plots
print("\nGenerating training plots...")
results.plot_results()
print("Training plots saved in the project directory")

# Print final summary
print("\n=== Export Summary ===")
print("The following files have been created:")
print(f"1. PyTorch model: {best_model_path}")
print(f"2. ONNX model: {best_model_path.replace('.pt', '.onnx')}")
print("3. Training plots: In the project directory")
print("\nModel is ready for deployment!")